In [1]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [97]:
!pip install git+https://github.com/harvardnlp/namedtensor

  Cloning https://github.com/harvardnlp/namedtensor to /tmp/pip-req-build-e93b4nwi
  Running command git clone -q https://github.com/harvardnlp/namedtensor /tmp/pip-req-build-e93b4nwi
     |████████████████████████████████| 71kB 8.3MB/s eta 0:00:011
     |████████████████████████████████| 71kB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 18.6MB/s eta 0:00:01
  Created wheel for namedtensor: filename=namedtensor-0.0.2-cp36-none-any.whl size=24820 sha256=e2bf54e702266faa18f5c3e6aa18e5030fc07c1e551c447194101132ab19f7c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-r6qe838o/wheels/67/14/88/b9602d277755e9dc045fe598dcf7ff0a0459740adc3ddb3660
Successfully built namedtensor


In [2]:
import models
import data
import utils
from domain import get_domain

In [98]:
from namedtensor import ntorch

In [25]:
import torch

In [8]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('hierarchical_rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder'
# model_fname = 'rel_tsel_ref_dial_model/default_0_ep-4.th'
# model is actually in "serialized_models/{model_fname}"

In [6]:
seed = 0
corpus = model_ty.corpus_ty(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_0.txt, total 544674, unks 4134, ratio 0.76%
dataset data/onecommon/valid_reference_0.txt, total 70004, unks 720, ratio 1.03%
dataset data/onecommon/test_reference_0.txt, total 67008, unks 576, ratio 0.86%


In [7]:
corpus

In [6]:
# model = utils.load_model(model_fname, map_location='cpu')

In [9]:
batchsize=10
validset, validset_stats = corpus.valid_dataset(batchsize)

In [10]:
validset_stats

{'n': 143700, 'self_n': 71834, 'nonpadn': 70004, 'self_nonpadn': 35002}

In [11]:
for bsz in [1, 2, 5, 10]:
    dset, dset_stats = corpus.valid_dataset(bsz)
    print(bsz, len(dset), dset_stats)

1 1038 {'n': 74862, 'self_n': 37431, 'nonpadn': 70004, 'self_nonpadn': 35002}
2 519 {'n': 95666, 'self_n': 47561, 'nonpadn': 70004, 'self_nonpadn': 35002}
5 214 {'n': 123064, 'self_n': 61796, 'nonpadn': 70004, 'self_nonpadn': 35002}
10 110 {'n': 143088, 'self_n': 71567, 'nonpadn': 70004, 'self_nonpadn': 35002}


In [12]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [13]:
for (input_vals, word_idxs, referent_idxs, output_idx, scenario_id, real_ids, agent, chat_id) in corpus.train:
    if 'C_d0de3d0144df405e998ae718539ff9a4' == chat_id:
        print('found it')
        break

ValueError: too many values to unpack (expected 8)

In [31]:
from collections import Counter

In [19]:
from argparse import Namespace

In [20]:
args = Namespace(only_first_mention=False)

In [18]:
def make_dots_mentioned(ref_tgt, args):
    assert ref_tgt.dim() == 3
    if args.only_first_mention:
        return ref_tgt[:,0,:] > 0
    else:
        return ref_tgt.sum(1) > 0


def make_dots_mentioned_multi(refs, args, bsz, num_dots):
    dots_mentioned = []
    for ref_tgt in refs:
        if ref_tgt is None:
            dots_mentioned.append(torch.zeros(bsz, num_dots).bool())
            continue
        dots_mentioned.append(make_dots_mentioned(ref_tgt, args))
    return dots_mentioned

In [23]:
bsz = 1
num_dots = 7

In [22]:
valid_dataset, valid_dataset_stats = corpus.valid_dataset(bsz)

In [16]:
ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, scenario_ids, real_ids, partner_real_ids, _, _, sel_idx, lens, rev_idxs, hid_idxs, num_markables, is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables = valid_dataset[0]

In [17]:
ctx

tensor([[-0.2750, -0.8050,  0.3333,  0.7200,  0.9250,  0.3600, -0.6667, -0.0800,
         -0.4000, -0.1650, -0.6667,  0.3467, -0.1000, -0.2550,  1.0000, -0.7200,
         -0.7350, -0.4300,  0.0000, -0.1867,  0.4100,  0.6550,  1.0000, -0.0533,
         -0.5800, -0.2400, -0.6667,  0.5600]])

In [26]:
dots_mentioned = make_dots_mentioned_multi(ref_tgts, args, bsz, num_dots)

In [27]:
dots_mentioned

[tensor([[False, False, False, False, False, False, False]]),
 tensor([[False, False,  True,  True, False, False,  True]]),
 tensor([[False, False, False, False, False, False, False]]),
 tensor([[False, False, False,  True, False, False, False]]),
 tensor([[False, False, False, False, False, False, False]])]

In [28]:
partner_dots_mentioned = make_dots_mentioned_multi(partner_ref_tgts_our_view, args, bsz, num_dots)

In [29]:
partner_dots_mentioned

[tensor([[ True, False, False, False, False, False, False]]),
 tensor([[False, False, False, False, False, False, False]]),
 tensor([[False, False, False,  True, False, False, False]]),
 tensor([[False, False, False, False, False, False, False]]),
 tensor([[False, False, False, False, False, False, False]])]

In [30]:
is_self

[tensor([False]),
 tensor([True]),
 tensor([False]),
 tensor([True]),
 tensor([False])]

In [102]:
# def tabulate_mentions(data):
#     # operate on batch 1
#     counts = Counter()
#     for instance in data:
#         ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, scenario_ids, real_ids, partner_real_ids, _, _, sel_idx, lens, rev_idxs, hid_idxs, num_markables, is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables = instance
#         dots_mentioned = make_dots_mentioned_multi(ref_tgts, args, bsz, num_dots)
#         partner_dots_mentioned = make_dots_mentioned_multi(partner_ref_tgts_our_view, args, bsz, num_dots)
#         for i in range(len(dots_mentioned)):
#             if not is_self[i].item():
#                 continue
#             dm = dots_mentioned[i].squeeze(0).tolist()
#             n1_pdm = partner_dots_mentioned[i-1].squeeze(0).tolist() if i > 0 else [None] * 7
#             n2_dm = dots_mentioned[i-2].squeeze(0).tolist() if i > 1 else [None] * 7
#             for dot in range(num_dots):
#                 key = (n2_dm[dot], n1_pdm[dot], dm[dot], i)
#                 counts[key] += 1
#     return counts

In [100]:
def val_to_index(val):
    if val is None:
        return 2
    elif val == False:
        return 0
    elif val == True:
        return 1
    else:
        raise Exception('invalid val {}'.format(val))

In [101]:
def index_to_val(index):
    return [False, True, None][index]

In [137]:
def tabulate_mentions(data):
    # operate on batch 1
    counts = ntorch.zeros(3, 3, 2, 12, 1, names=("selfprev", "partner", "self", "turn", "value"))
    for instance in data:
        ctx, inpts, tgts, ref_inpts, ref_tgts, sel_tgt, scenario_ids, real_ids, partner_real_ids, _, _, sel_idx, lens, rev_idxs, hid_idxs, num_markables, is_self, partner_ref_inpts, partner_ref_tgts_our_view, all_partner_num_markables = instance
        dots_mentioned = make_dots_mentioned_multi(ref_tgts, args, bsz, num_dots)
        partner_dots_mentioned = make_dots_mentioned_multi(partner_ref_tgts_our_view, args, bsz, num_dots)
        for i in range(len(dots_mentioned)):
            if not is_self[i].item():
                continue
            dm = dots_mentioned[i].squeeze(0).tolist()
            n1_pdm = partner_dots_mentioned[i-1].squeeze(0).tolist() if i > 0 else [None] * 7
            n2_dm = dots_mentioned[i-2].squeeze(0).tolist() if i > 1 else [None] * 7
            for dot in range(num_dots):
                counts[{
                    'selfprev': val_to_index(n2_dm[dot]),
                    'partner': val_to_index(n1_pdm[dot]),
                    'self': val_to_index(dm[dot]),
                    'turn': i,
                }] += 1
    return counts

In [138]:
counts = ntorch.zeros(3, 3, 2, 12, 1, names=("selfprev", "partner", "self", "turn", "value"))

In [141]:
mention_counts = tabulate_mentions(valid_dataset)

In [142]:
# def filter_key(key, indices_to_keep):
#     new_key = tuple(x for i, x in enumerate(key) if i in indices_to_keep)
#     return new_key

In [90]:
# def marginalize(counter, indices_to_keep):
#     new_counter = Counter()
#     for key, value in counter.items():
#         new_key = filter_key(key, indices_to_keep)
#         new_counter[new_key] += value
#     return new_counter

In [155]:
# def conditional(counter, conditioning_indices):
#     marginal = marginalize(counter, conditioning_indices)
#     conditionals = {}
#     for key, value in counter.items():
#         context = filter_key(key, conditioning_indices)
#         denom = marginal[context]
#         conditionals[key] = value / denom
#     return conditionals

In [165]:
def marginalize(counter, dims_to_keep):
    dims_to_keep = set(dims_to_keep).copy()
    dims_to_keep.add("value")
    dims = counter._schema._names
    assert all(d in dims for d in dims_to_keep), (dims, dims_to_keep)
    to_marginalize = set(dims) - set(dims_to_keep)
    c = counter
    for d in to_marginalize:
        c = c.sum(d)
    return c

In [166]:
def conditional(counter, conditioning_dims):
    marginals = marginalize(counter, conditioning_dims)
    return counter.float() / marginals

In [167]:
mention_counts

NamedTensor(
	tensor([[[[[0.0000e+00],
           [0.0000e+00],
           [1.6820e+03],
           [1.4470e+03],
           [1.0000e+03],
           [6.4200e+02],
           [3.6700e+02],
           [1.7400e+02],
           [5.4000e+01],
           [3.4000e+01],
           [1.7000e+01],
           [1.2000e+01]],

          [[0.0000e+00],
           [0.0000e+00],
           [1.7300e+02],
           [1.2400e+02],
           [6.6000e+01],
           [3.0000e+01],
           [2.5000e+01],
           [1.3000e+01],
           [1.0000e+00],
           [0.0000e+00],
           [2.0000e+00],
           [0.0000e+00]]],


         [[[0.0000e+00],
           [0.0000e+00],
           [1.9200e+02],
           [1.2000e+02],
           [9.4000e+01],
           [5.6000e+01],
           [2.0000e+01],
           [3.6000e+01],
           [1.0000e+01],
           [4.0000e+00],
           [0.0000e+00],
           [2.0000e+00]],

          [[0.0000e+00],
           [0.0000e+00],
           [2.0700e+02],
   

In [168]:
marginalize(mention_counts, ["self"])

NamedTensor(
	tensor([[12833.],
        [ 4170.]]),
	('self', 'value'))

In [169]:
conditional(marginalize(mention_counts, ["self"]), [])

NamedTensor(
	tensor([[0.7547],
        [0.2453]]),
	('self', 'value'))

In [93]:
# p(next_mention)
conditional(marginalize(mention_counts, [2]), [])

{(False,): 0.7547491619126037, (True,): 0.24525083808739634}

In [94]:
# p(next_mention | this_partner_mention)
conditional(marginalize(mention_counts, [1,2]), [0])

{(True, False): 0.5078389179219183,
 (False, False): 0.8673519818127904,
 (False, True): 0.13264801818720964,
 (True, True): 0.4921610820780818,
 (None, False): 0.662262592898431,
 (None, True): 0.33773740710156896}

In [170]:
conditional(marginalize(mention_counts, ["partner", "self"]), ["partner"])

NamedTensor(
	tensor([[[0.8674],
         [0.1326]],

        [[0.5078],
         [0.4922]],

        [[0.6623],
         [0.3377]]]),
	('partner', 'self', 'value'))

In [173]:
vars(mention_counts._schema)

{'_names': ('selfprev', 'partner', 'self', 'turn', 'value'),
 '_masked': 0,
 '_axes': OrderedDict([('selfprev', 0),
              ('partner', 1),
              ('self', 2),
              ('turn', 3),
              ('value', 4)])}

In [175]:
mention_counts._tensor.size()

torch.Size([3, 3, 2, 12, 1])

In [176]:
import itertools

In [177]:
itertools.product?

In [216]:
def query(tensor, query_vars, conditioning_vars=None):
    if conditioning_vars is None:
        conditioning_vars = {}
    query_vars = list(query_vars).copy()
    conditioning_vars = list(conditioning_vars)
    assert all(v not in conditioning_vars for v in query_vars)
    all_vars = set(query_vars) | set(conditioning_vars)
    marg = marginalize(tensor, all_vars)
    cond = conditional(marg, conditioning_vars)
    return cond

In [299]:
def display_query(tensor, query_vars, conditioning_vars=None):
    if conditioning_vars is None:
        conditioning_vars = {}
    query_vars = list(query_vars).copy()
    conditioning_vars = list(conditioning_vars)
    assert all(v not in conditioning_vars for v in query_vars)
    all_vars = set(query_vars) | set(conditioning_vars)
    marg = marginalize(tensor, all_vars)
    prior_probs = conditional(marginalize(tensor, conditioning_vars), {})
    cond = conditional(marg, conditioning_vars)
    
    ranges = {v: list(range(marg._tensor.size(marg._schema._axes[v]))) for v in all_vars}
    conditioning_its = [ranges[v] for v in conditioning_vars]
    query_its = [ranges[v] for v in query_vars]
    query_vars = query_vars + ['value']
    query_its = query_its + [list(range(1))]
    expected_acc = 0
    for conditioning_key in itertools.product(*conditioning_its):
        cond_ix = {
            var: index
            for var, index in zip(conditioning_vars, conditioning_key)
        }
        for query_key in itertools.product(*query_its):
            query_ix = {
                var: index
                for var, index in zip(query_vars, query_key)
            }
            val = cond[{
                **cond_ix,
                **query_ix,
            }].item()
            query_str = ', '.join('{}={}'.format(var, val) for (var, val) in zip(query_vars, query_key) if var != 'value')
            cond_str = ', '.join('{}={}'.format(var, val) for (var, val) in zip(conditioning_vars, conditioning_key))
            if cond_str:
                print("p({} | {}) =  {}".format(query_str, cond_str, val))
            else:
                print("p({}) = {}".format(query_str, val))
        if len(query_vars) == 2:
            qv = query_vars[0]
            max_acc, max_ix = cond[cond_ix].max(qv)
            cond_expected_acc = max_acc[{'value': 0}].item()
            prior_prob = prior_probs[{**cond_ix}].item()
            print("expected acc: {}; prior prob: {}".format(cond_expected_acc, prior_prob))
            if prior_prob > 0:
                expected_acc += cond_expected_acc * prior_prob
    if len(query_vars) == 2:
        print("expected accuracy: {}".format(expected_acc))

In [300]:
display_query(mention_counts, {"self"})

p(self=0) = 0.7547491788864136
p(self=1) = 0.24525083601474762
expected acc: 0.7547491788864136; prior prob: 1.0
expected accuracy: 0.7547491788864136


In [301]:
display_query(mention_counts, {"self"}, {"partner"})
cond = query(mention_counts, {"self"}, {"partner"})

p(self=0 | partner=0) =  0.8673520088195801
p(self=1 | partner=0) =  0.1326480209827423
expected acc: 0.8673520088195801; prior prob: 0.5950126647949219
p(self=0 | partner=1) =  0.5078389048576355
p(self=1 | partner=1) =  0.4921610951423645
expected acc: 0.5078389048576355; prior prob: 0.19131918251514435
p(self=0 | partner=2) =  0.6622626185417175
p(self=1 | partner=2) =  0.33773741126060486
expected acc: 0.6622626185417175; prior prob: 0.21366818249225616
expected accuracy: 0.754749204246087


In [302]:
display_query(mention_counts, {"self"}, {"partner", "selfprev"})

p(self=0 | partner=0, selfprev=0) =  0.9259764552116394
p(self=1 | partner=0, selfprev=0) =  0.07402353733778
expected acc: 0.9259764552116394; prior prob: 0.3448215126991272
p(self=0 | partner=0, selfprev=1) =  0.775522768497467
p(self=1 | partner=0, selfprev=1) =  0.22447724640369415
expected acc: 0.775522768497467; prior prob: 0.09563018381595612
p(self=0 | partner=0, selfprev=2) =  0.793379008769989
p(self=1 | partner=0, selfprev=2) =  0.20662100613117218
expected acc: 0.793379008769989; prior prob: 0.15456095337867737
p(self=0 | partner=1, selfprev=0) =  0.5603357553482056
p(self=1 | partner=1, selfprev=0) =  0.43966421484947205
expected acc: 0.5603357553482056; prior prob: 0.056048933416604996
p(self=0 | partner=1, selfprev=1) =  0.5407066345214844
p(self=1 | partner=1, selfprev=1) =  0.459293395280838
expected acc: 0.5407066345214844; prior prob: 0.07657472044229507
p(self=0 | partner=1, selfprev=2) =  0.41482967138290405
p(self=1 | partner=1, selfprev=2) =  0.585170328617096
ex

In [304]:
display_query(mention_counts, {"self"}, {"partner", "selfprev", "turn"})

p(self=0 | turn=0, partner=0, selfprev=0) =  nan
p(self=1 | turn=0, partner=0, selfprev=0) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=0, selfprev=1) =  nan
p(self=1 | turn=0, partner=0, selfprev=1) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=0, selfprev=2) =  nan
p(self=1 | turn=0, partner=0, selfprev=2) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=1, selfprev=0) =  nan
p(self=1 | turn=0, partner=1, selfprev=0) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=1, selfprev=1) =  nan
p(self=1 | turn=0, partner=1, selfprev=1) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=1, selfprev=2) =  nan
p(self=1 | turn=0, partner=1, selfprev=2) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=2, selfprev=0) =  nan
p(self=1 | turn=0, partner=2, selfprev=0) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=0, partner=2, selfprev=1) =  nan
p(self=1 | turn=0, p

expected acc: 0.6818181872367859; prior prob: 0.00129388936329633
p(self=0 | turn=8, partner=0, selfprev=2) =  nan
p(self=1 | turn=8, partner=0, selfprev=2) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=8, partner=1, selfprev=0) =  0.7142857313156128
p(self=1 | turn=8, partner=1, selfprev=0) =  0.2857142984867096
expected acc: 0.7142857313156128; prior prob: 0.0008233840926550329
p(self=0 | turn=8, partner=1, selfprev=1) =  1.0
p(self=1 | turn=8, partner=1, selfprev=1) =  0.0
expected acc: 1.0; prior prob: 0.00041169204632751644
p(self=0 | turn=8, partner=1, selfprev=2) =  nan
p(self=1 | turn=8, partner=1, selfprev=2) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=8, partner=2, selfprev=0) =  nan
p(self=1 | turn=8, partner=2, selfprev=0) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=8, partner=2, selfprev=1) =  nan
p(self=1 | turn=8, partner=2, selfprev=1) =  nan
expected acc: nan; prior prob: 0.0
p(self=0 | turn=8, partner=2, selfprev=2) =  nan
p(sel